In [86]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import re

from tqdm import tqdm

random_state = 12345

import json
import os

In [87]:
train_ds = pd.read_csv('data/train_dataset.csv')
train_ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37224 entries, 0 to 37223
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          37224 non-null  object 
 1   time          37224 non-null  int64  
 2   target        37224 non-null  float64
 3   temp          37224 non-null  float64
 4   temp_pred     37136 non-null  float64
 5   weather_pred  37136 non-null  object 
 6   weather_fact  37223 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 2.0+ MB


In [88]:
train_ds['date'] = pd.to_datetime(train_ds['date'])
train_ds['year'] = train_ds['date'].dt.year
train_ds['month'] = train_ds['date'].dt.month
train_ds['day_of_week'] = train_ds['date'].dt.dayofweek
train_ds['day'] = train_ds['date'].dt.day
train_ds['day_of_year'] = train_ds['date'].dt.dayofyear

In [89]:
# функция формирует колонки 'cloudy', 'rainy', 'windy', 'clear', 'some_number'
# в колонках число, которое 0 при отсутсвии упоминания явления в weather_pred или степень упоминания
# функция дает в колонках номер первого списка, элемент которого есть в строке плюс 1
# списки cloudy_list, rainy_list, windy_list, clear_list можно модифицировать
# соответственно, можно экспериментировать с расположением значений в списках
# например, сейчас 'дождь', 'снег', 'д+сн' - первая степень  дождя, а 'гроз', 'ливень' - вторая
# а можно сделать снег второй, а грозу с ливнем убрать в третью
# также сделал отдельный список для "ясности", чтобы выделить 'ясно' и 'солнечно'

def in_what_list(weather, big_list):
    for list_number, small_list in enumerate(big_list):
        if any(word in weather for word in small_list):
            return list_number+1
    return 0

def weather_split2(row):
    weather = row['weather_pred']
    cloudy_list = [['проясн', 'пер.об.', 'п/об'], ['пасм', 'обл']]
    rainy_list = [['дождь', 'снег', 'д+сн'], ['гроз', 'ливень']]
    windy_list = [['вет'],['штор']]
    clear_list = [['проясн'], ['ясно'], ['солнеч']]
    numbers = re.findall(r'\d+', weather)
    cloudy = in_what_list(weather, cloudy_list)
    rainy = in_what_list(weather, rainy_list)
    windy = in_what_list(weather, windy_list)
    clear = in_what_list(weather, clear_list)
    rain_probability = 0 if len(numbers)==0 else int(numbers[0])
    has_rain_probability = int(len(numbers)==0)
    return cloudy, rainy, windy, clear, rain_probability, has_rain_probability

def fill_weather_columns(df):
    df['weather_pred'] = df['weather_pred'].fillna('')
    df['cloudy'], df['rainy'], df['windy'], df['clear'], df['rain_probability'], df['has_rain_probability'] = \
                zip(*df.apply(weather_split2, axis=1))
    return df

train_ds = fill_weather_columns(train_ds)



In [90]:
#tmpds[(tmpds['cloudy']==0) & (tmpds['clear']==0) & (tmpds['rainy']==0)]['weather_pred'].value_counts()

In [91]:
train_ds.columns

Index(['date', 'time', 'target', 'temp', 'temp_pred', 'weather_pred',
       'weather_fact', 'year', 'month', 'day_of_week', 'day', 'day_of_year',
       'cloudy', 'rainy', 'windy', 'clear', 'rain_probability',
       'has_rain_probability'],
      dtype='object')

In [92]:
train_ds.head()

,date,time,target,temp,temp_pred,weather_pred,weather_fact,year,month,day_of_week,day,day_of_year,cloudy,rainy,windy,clear,rain_probability,has_rain_probability
0,2019-01-01,0,481.510,2.9,2.0,"пасм, ветер",ветер,2019,1,1,1,1,2,0,1,0,0,1
1,2019-01-01,1,462.872,2.9,2.0,"пасм, ветер",ветер,2019,1,1,1,1,2,0,1,0,0,1
2,2019-01-01,2,449.718,2.9,2.0,"пасм, ветер",ветер,2019,1,1,1,1,2,0,1,0,0,1
3,2019-01-01,3,430.908,4.3,2.0,"пасм, ветер","ветер, пасм",2019,1,1,1,1,2,0,1,0,0,1
4,2019-01-01,4,415.163,4.3,2.0,"пасм, ветер","ветер, пасм",2019,1,1,1,1,2,0,1,0,0,1


In [93]:
folder_path = 'data/celebrates'

# Получаем список всех файлов JSON в папке
json_files = [file for file in os.listdir(folder_path) if file.endswith('.json')]

# Создаем пустой датафрейм для хранения данных
df_holidays = pd.DataFrame()

# Проходим по каждому файлу JSON
for json_file in json_files:
    # Открываем файл и загружаем данные
    with open(os.path.join(folder_path, json_file), 'r') as file:
        data = json.load(file)
    
    # Преобразуем списки в датафреймы
    df_holidays = pd.DataFrame(data['holidays'], columns=['date'])
    df_preholidays = pd.DataFrame(data['preholidays'], columns=['date'])

    # Добавляем столбцы для праздников и предпраздничных дней
    df_holidays['holidays'] = 1
    df_preholidays['preholidays'] = 1

    # Объединяем датафреймы
    temp_df = pd.concat([df_holidays, df_preholidays])

    # Заполняем пропущенные значения нулями
    temp_df.fillna(0, inplace=True)

    # Добавляем временный датафрейм в общий датафрейм
    df_holidays = pd.concat([df_holidays, temp_df])

# Преобразуем столбец 'date' в формат datetime
df_holidays['date'] = pd.to_datetime(df_holidays['date'])

df_holidays.fillna(0, inplace=True)
df_holidays.to_csv('data/holidays.csv', index=False)
# Выводим первые строки датафрейма
print(df_holidays.sample(10))

         date  holidays  preholidays
97 2020-10-18       1.0          0.0
47 2020-05-09       1.0          0.0
63 2020-06-27       1.0          0.0
82 2020-08-29       1.0          0.0
94 2020-10-10       1.0          0.0
78 2020-08-15       1.0          0.0
59 2020-06-13       1.0          0.0
63 2020-06-27       1.0          0.0
6  2020-01-07       1.0          0.0
64 2020-06-28       1.0          0.0


In [94]:
# Assuming df_holidays and train_ds are your dataframes
train_ds = pd.merge(train_ds, df_holidays, on='date', how='left')

# Fill NaN values with 0
train_ds['holidays'].fillna(0, inplace=True)
train_ds['preholidays'].fillna(0, inplace=True)

# Convert to int
train_ds['holidays'] = train_ds['holidays'].astype(int)
train_ds['preholidays'] = train_ds['preholidays'].astype(int)

In [95]:
train_ds.head()

,date,time,target,temp,temp_pred,weather_pred,weather_fact,year,month,day_of_week,day,day_of_year,cloudy,rainy,windy,clear,rain_probability,has_rain_probability,holidays,preholidays
0,2019-01-01,0,481.510,2.9,2.0,"пасм, ветер",ветер,2019,1,1,1,1,2,0,1,0,0,1,0,0
1,2019-01-01,1,462.872,2.9,2.0,"пасм, ветер",ветер,2019,1,1,1,1,2,0,1,0,0,1,0,0
2,2019-01-01,2,449.718,2.9,2.0,"пасм, ветер",ветер,2019,1,1,1,1,2,0,1,0,0,1,0,0
3,2019-01-01,3,430.908,4.3,2.0,"пасм, ветер","ветер, пасм",2019,1,1,1,1,2,0,1,0,0,1,0,0
4,2019-01-01,4,415.163,4.3,2.0,"пасм, ветер","ветер, пасм",2019,1,1,1,1,2,0,1,0,0,1,0,0


In [96]:
# создаем столбец 'temp_last_day'
train_ds['temp_last_day'] = train_ds['temp'].shift(24) # погода сутки назад
# Среднюю температуру сутки назад
# Максимальную температуру сутки назад
# Минимальный  температуру сутки назад
# заполняем пропущенные значения в 'temp_last_day'
train_ds['temp_last_day'].fillna(method='bfill', inplace=True)

# создаем столбцы с временными лагами для 'target'
lags = [24, 48, 72, 7*24, 14*24]
for lag in lags:
    train_ds[f'target_lag_{lag}'] = train_ds['target'].shift(lag)

# заполняем пропущенные значения в столбцах с лагами
for lag in lags:
    train_ds[f'target_lag_{lag}'].fillna(0, inplace=True)

/var/folders/4y/zccbjjq17fgd73999h5g3ltr0000gn/T/ipykernel_8747/1739306973.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_ds['temp_last_day'].fillna(method='bfill', inplace=True)


In [97]:
train_ds = train_ds.drop('temp', axis=1)

In [98]:
train_ds.head()

,date,time,target,temp_pred,weather_pred,weather_fact,year,month,day_of_week,day,...,rain_probability,has_rain_probability,holidays,preholidays,temp_last_day,target_lag_24,target_lag_48,target_lag_72,target_lag_168,target_lag_336
0,2019-01-01,0,481.510,2.0,"пасм, ветер",ветер,2019,1,1,1,...,0,1,0,0,2.9,0.0,0.0,0.0,0.0,0.0
1,2019-01-01,1,462.872,2.0,"пасм, ветер",ветер,2019,1,1,1,...,0,1,0,0,2.9,0.0,0.0,0.0,0.0,0.0
2,2019-01-01,2,449.718,2.0,"пасм, ветер",ветер,2019,1,1,1,...,0,1,0,0,2.9,0.0,0.0,0.0,0.0,0.0
3,2019-01-01,3,430.908,2.0,"пасм, ветер","ветер, пасм",2019,1,1,1,...,0,1,0,0,2.9,0.0,0.0,0.0,0.0,0.0
4,2019-01-01,4,415.163,2.0,"пасм, ветер","ветер, пасм",2019,1,1,1,...,0,1,0,0,2.9,0.0,0.0,0.0,0.0,0.0


In [99]:
feature_cols = list(train_ds.columns)
drop_list = ['target', 'date', 'day_of_year', 'weather_pred', 'weather_fact']
for name in drop_list:
    feature_cols.remove(name)

feature_cols

['time',
 'temp_pred',
 'year',
 'month',
 'day_of_week',
 'day',
 'cloudy',
 'rainy',
 'windy',
 'clear',
 'rain_probability',
 'has_rain_probability',
 'holidays',
 'preholidays',
 'temp_last_day',
 'target_lag_24',
 'target_lag_48',
 'target_lag_72',
 'target_lag_168',
 'target_lag_336']

In [59]:
def mae_day(y_true, y_pred):
    y_true_copy = pd.DataFrame(y_true).reset_index(drop=True)
    y_true_copy['day'] = y_true_copy.index // 24
    y_true_grouped = y_true_copy.groupby(by='day').sum()   
    y_pred_copy = pd.DataFrame(y_pred).reset_index(drop=True)
    y_pred_copy['day'] = y_pred_copy.index // 24
    y_pred_grouped = y_pred_copy.groupby(by='day').sum()
    
    return mean_absolute_error(y_true_grouped, y_pred_grouped)

In [33]:
!pip install statsmodels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 9.6 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.8/233.8 kB 15.2 MB/s eta 0:00:00


In [79]:
# импортируем SARIMAX
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller

In [101]:
feature_cols = list(train_ds.columns)
print(feature_cols)
drop_list = ['date', 'day_of_year', 'weather_pred', 'weather_fact']
for name in drop_list:
    feature_cols.remove(name)

['date', 'time', 'target', 'temp_pred', 'weather_pred', 'weather_fact', 'year', 'month', 'day_of_week', 'day', 'day_of_year', 'cloudy', 'rainy', 'windy', 'clear', 'rain_probability', 'has_rain_probability', 'holidays', 'preholidays', 'temp_last_day', 'target_lag_24', 'target_lag_48', 'target_lag_72', 'target_lag_168', 'target_lag_336']


In [102]:
train_ds_sarrima = train_ds[feature_cols]
train_ds_sarrima.tail()

,time,target,temp_pred,year,month,day_of_week,day,cloudy,rainy,windy,...,rain_probability,has_rain_probability,holidays,preholidays,temp_last_day,target_lag_24,target_lag_48,target_lag_72,target_lag_168,target_lag_336
40075,19,552.960,6.0,2023,3,4,31,2,0,0,...,61,0,0,0,7.8,561.542,565.138,590.618,559.747,593.441
40076,20,563.985,6.0,2023,3,4,31,2,0,0,...,61,0,0,0,7.8,582.464,586.935,609.268,570.206,615.636
40077,21,560.191,5.0,2023,3,4,31,2,0,0,...,61,0,0,0,7.1,571.899,578.999,596.068,555.419,594.370
40078,22,538.796,5.0,2023,3,4,31,2,0,0,...,61,0,0,0,7.1,545.723,555.065,568.961,530.734,569.087
40079,23,518.685,5.0,2023,3,4,31,2,0,0,...,61,0,0,0,7.1,520.055,529.184,545.832,511.163,545.555


In [103]:
target = train_ds_sarrima['target']

# Выполните тест Дики-Фуллера
result = adfuller(target)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistic: -4.009688
p-value: 0.001361
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567


##### Эти данные - результаты теста Дики-Фуллера, который используется для проверки стационарности временного ряда.

ADF Statistic: Это значение статистики теста. Более отрицательное значение указывает на то, что временной ряд скорее всего является стационарным.

p-value: Это вероятность того, что нулевая гипотеза (временной ряд нестационарен) верна. Если p-value меньше 0.05, мы отвергаем нулевую гипотезу и считаем временной ряд стационарным.

Critical Values: Это значения, которые используются для сравнения со значением статистики теста. Если ADF Statistic меньше (по модулю) критического значения, мы отвергаем нулевую гипотезу.

В вашем случае, ADF Statistic равно -4.009688, что меньше (по модулю) критического значения на уровне 1% (-3.431). Кроме того, p-value равно 0.001361, что меньше 0.05. Это означает, что вы можете отвергнуть нулевую гипотезу и считать ваш временной ряд стационарным.


order=(p, d, q): Это параметры для нерегулярной части модели ARIMA

p: порядок авторегрессии (AR). Это количество лаговых наблюдений в модели 

d: порядок интеграции (I). Это порядок разности, применяемый к временному ряду для достижения стационарности 

q: порядок скользящего среднего (MA)

Выбираем d = 0

In [ ]:
ARIMA_d=0